In [2]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [3]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lendingDF = pd.read_csv('lending_data.csv')

# Review the DataFrame
lendingDF.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [4]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lendingDF['loan_status']

# Separate the X variable, the features
X = lendingDF.drop(columns='loan_status')

In [5]:
# Review the y variable Series (first five rows)
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [6]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [7]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [8]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
x_test, x_train, y_test, y_train = train_test_split(X,y, random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [9]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
log_reg_model = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
log_reg_model.fit(x_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [10]:
# Make a prediction using the testing data
predictions = log_reg_model.predict(x_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [12]:
# Print the balanced_accuracy score of the model
print(f"Balanced Accuracy Score: {balanced_accuracy_score(y_test,predictions)*100:.2f}%")

Balanced Accuracy Score: 94.31%


In [14]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_test, predictions))

[[55999   272]
 [  205  1676]]


In [15]:
# Print the classification report for the model
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56271
           1       0.86      0.89      0.88      1881

    accuracy                           0.99     58152
   macro avg       0.93      0.94      0.94     58152
weighted avg       0.99      0.99      0.99     58152



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Looking at the top portion of the classification report and confusion matrix, we can see that the precision and recall for the healthy loans were extremely high at almost 100%. However, the model did not perform as well on labeling the high-risk loans. The high risk loans had about 86 percent precision (1676/(1672+272)) and 89 percent recall (1676/(1672+205)).

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points.

In [21]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros_model = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = ros_model.fit_resample(x_train, y_train)

In [22]:
# Count the distinct values of the resampled labels data
y_resampled

0        0
1        0
2        0
3        0
4        0
        ..
37525    1
37526    1
37527    1
37528    1
37529    1
Name: loan_status, Length: 37530, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [24]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
log_reg_model_resampled = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
log_reg_model_resampled.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
predictedValues_resampled = log_reg_model_resampled.predict(x_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [25]:
# Print the balanced_accuracy score of the model
print(f"Balanced Accuracy Score (resampled): {balanced_accuracy_score(y_test, predictedValues_resampled)*100:.2f}%")

Balanced Accuracy Score (resampled): 99.49%


In [26]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_test, predictedValues_resampled))

[[55964   307]
 [    9  1872]]


In [27]:
# Print the classification report for the model
print(classification_report(y_test, predictedValues_resampled))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     56271
           1       0.86      1.00      0.92      1881

    accuracy                           0.99     58152
   macro avg       0.93      0.99      0.96     58152
weighted avg       1.00      0.99      0.99     58152



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** This time around, we did a much better job of predicting the risky loans!

Looking at the top portion of the classification report and confusion matrix, we can see that the precision and recall for the healthy loans were still extremely high at about 100% and 99%, respectively. But more importantly, the model was able to improve it's evaluation of high risk loans. Precision stayed at about 86% precision (1872/(1872+307)), but the recall improved to almost 100% (1872/(1872+9))!

Ultimately, the resampled model is more cautious model, and would be a better model to use for identifying potentially risky loans, since it was able to reduce the number of loans falsely identified as healthy!